# Least-squares regression exercise




## Introduction and definitions

In [1]:
#: Import numerical and plotting libraries
import numpy as np
# Print to four digits of precision
np.set_printoptions(precision=4, suppress=True)
import numpy.linalg as npl
import matplotlib.pyplot as plt

These exercises are to practice thinking about how the regression estimation
works, and the relationship of correlation and regression.

To give us some concrete data to play with, here are another couple of samples
of the “psychopathy” and “clamminess” scores, of the same type that we saw in
the [introduction to the general linear
model](https://textbook.nipraxis.org/glm_intro.html):

In [2]:
#: The data, that we are trying to model.
psychopathy = np.array([ 11.914,   4.289,  10.825,  14.987,
                         7.572,   5.447,   17.332,  12.105,
                         13.297,  10.635,  21.777,  20.715])

In [3]:
#: The regressor that we will use to model the data.
clammy = np.array([ 0.422,  0.406,  0.061,  0.962,  4.715,
                    1.398,  1.952,  5.095, 8.092,  5.685,
                    5.167,  7.257])

$\newcommand{\yvec}{\vec{y}} \newcommand{\xvec}{\vec{x}} \newcommand{\evec}{\vec{\varepsilon}}$

Our simple linear model can be expressed by:

$$
y_i = c + bx_i + e_i`
$$

In vector notation:

$$
\yvec = c + b \xvec + \evec
$$

$\yvec$ is the vector of values $[y_1, y_2, ... y_n]$ we want to explain
(psychopathy), $\xvec$ is the vector of values $[x_1, x_2, ... x_n]$ containing
our explanatory variable (clammy), and $\evec$ is the vector of remaining data
unexplained by $c + b \xvec$.

$\newcommand{Xmat}{\boldsymbol X} \newcommand{\bvec}{\vec{\beta}}$

The same model can also be expressed using a design *matrix* $\Xmat$:

$$
\yvec = \Xmat \bvec + \evec
$$

where $\Xmat$ has two columns, the first being a vector of $n$ ones, and the
second being $\xvec$. $\bvec$ is a column vector containing two values, $[c,
b]$ that are, respectively, the intercept and slope of the fitted line.

Now define the *mean* of $\vec{x}$ as:

$$
\bar{x} = \frac{1}{n} \sum_{i=1}^n x_i
$$

Define two new vectors, $\vec{x^c}, \vec{y^c}$ that contain the values in
$\vec{x}, \vec{y}$ with their respective means subtracted:

$$
\vec{x^c} = [x_1 - \bar{x}, x_2 - \bar{x}, ... , x_n - \bar{x}]

\vec{y^c} = [y_1 - \bar{y}, y_2 - \bar{y}, ... , y_n - \bar{y}]
$$

We found in [introduction to the general linear
model](https://textbook.nipraxis.org/glm_intro.html) that, for the
case of a [full-rank](https://textbook.nipraxis.org) matrix $\Xmat$, the least
squares estimate for $\bvec$ is given by:

$$
\newcommand{\bhat}{\hat{\bvec}} \newcommand{\yhat}{\hat{\yvec}}
\bhat = (\Xmat^T \Xmat)^{-1} \Xmat^T \yvec
$$

## On vector lengths

In what follows, we will refer to the idea of the *vector length* of a vector.
This is different from the number of values in a vector, often written as $n$:

In [4]:
n = len(clammy)
n

In contrast *vector length* is a mathematical property of the vector *values*.
See [vectors and dot
products](http://matthew-brett.github.io/teaching/on_vectors.html) for the
mathematical background.  In summary, vector length is a generalization of
Pythagoras' theorem to more than two dimensions, and is defined as the square
root of the sum of the squared vector values.  For example, the vector length
of `clammy` is:

In [5]:
np.sqrt(np.sum(clammy ** 2))

Now consider a vector of values for which the mean value is 0.   We can create
such a vector by subtracting the mean from the vector values, like this:

In [6]:
x_1 = clammy - np.mean(clammy)
# The mean is (near as the computer can calculate) 0
np.mean(x_1)

The length of this vector is:

In [7]:
x_1_len = np.sqrt(np.sum(x_1 ** 2))
x_1_len

Here is the *standard deviation* of this vector:

In [8]:
x_1_std = np.std(x_1)
x_1_std

Thinking about the calculation of the standard deviation and the calculation of
the length, how would you recalculate the standard deviation *from the length*?

You may well need a sheet of paper to work out how to do this, with some
algebra.

In [9]:
# Recalculate the standard deviation from the vector length.
std_recalculated = np.sqrt(x_1_len ** 2 / n)
# Show the result
std_recalculated

Conversely, calculate the vector length from the standard deviation:

In [10]:
# Recalculate the vector length from the standard deviation
vlen_recalculated = np.sqrt(x_1_std ** 2 * n)
# Show the result
vlen_recalculated

## On dot products

The *dot product* is the sum of the values in a vector that results from
multiplying the two vectors together, elementwise.

For example, to calculate the dot product of `psychopathy` and `clammy`, we
first multiply the two vectors together, elementwise:

In [11]:
multiplied = clammy * psychopathy
multiplied

The dot product is the sum of the values in `multiplied`:

In [12]:
np.sum(multiplied)

And in general:

In [13]:
def dot_product(x, y):
    return np.sum(x * y)

In [14]:
dot_product(clammy, psychopathy)

Numpy also has a short-cut for the dot-product operation, because it is so
common:

In [15]:
np.dot(clammy, psychopathy)

In fact, this is what you get when using the `@` operator in Numpy, called the
*matrix multiplication* operator, when applied to two vectors:

In [16]:
clammy @ psychopathy

Have a look at the definitions of *vector length* and the dot product.  How
would you calculate the vector length from the dot product of the vector with
itself?

In [17]:
#- Recalculate the vector length using the dot product of the vector with itself
c_dot_c = clammy @ clammy
np.sqrt(c_dot_c)

## Correlation coefficient and regression

Create the $\Xmat$ matrix from a vector of ones and the vector of `clammy`
scores:

In [18]:
#- Create X design matrix from column of ones and clammy vector
X = np.stack([np.ones(n), clammy], axis=1)
# Show the result.
X

Are the columns of `X` orthogonal to each other?

Two *vectors* are defined as being orthogonal to each other if the *dot
product* of the two vectors is zero.

In [19]:
#- Are the two columns of X orthogonal?
X[:, 0] @ X[:, 1]

We will now calculate the pseudoinverse.

First we calculate $\Xmat^T \Xmat$:

In [20]:
XtX = X.T @  X
XtX

*For extra points* : does this matrix help us with the question as to whether
the columns are orthogonal?  How?

Calculated the matrix inverse of $\Xmat^T \Xmat$:

In [21]:
#- Matrix inverse of X.T @ X
iXtX = npl.inv(XtX)
# Show the result
iXtX

If you can calculate the inverse without error, that means that $\Xmat^T \Xmat$
is *invertible*.

Calculate $(\Xmat^T \Xmat)^{-1} \Xmat^T$ (the pseudoinverse in the invertible
case).  What shape is it?

In [22]:
#- Calculate (X.T X)^-1 X.T (the pseudoinverse)
piX = iXtX @ X.T
# Show the shape
piX.shape

Calculate the least squares fit value for $\bvec$:

In [23]:
#- Calculate least squares fit for beta vector
B = piX @ psychopathy
B

Calculate the fitted values $c + b \xvec$, and the residuals $\evec$:

In [24]:
#- Calculate the fitted values
fitted = X @ B
residuals = psychopathy - fitted

Confirm that the mean of the residuals is close to zero:

In [25]:
#- mean of residuals near zero
np.allclose(np.mean(residuals), 0)

Confirm that residuals are orthogonal to both columns of the design matrix:

In [26]:
#- Show residuals orthogonal to both columns of design
np.allclose(X.T @ residuals, 0)

We will now modify the design to see what happens to the parameters and the
fitted values.

We build the new design by leaving the first columns the same (a column of
ones).   Then we change the second column (the `clammy` values) so that it is
*orthogonal to* the first.  We can do that by subtracting the mean from the
`clammy` vector:

In [27]:
x_1 = clammy - np.mean(clammy)

Confirm that `x_1` is orthogonal to the column of ones:

In [28]:
#- Show x_1 orthogonal to a column of ones
x_1 @ np.ones(n)

Compile the column of ones and the new regressor into an `n` by 2 design matrix
`X_o`:

In [29]:
X_o = np.stack([np.ones(n), x_1], axis=1)
# Show the result
X_o

Here is the matrix that we will invert to make the pseudoinverse:

In [30]:
XtX_o = X_o.T @ X_o
XtX_o

Look at the diagonal values of the matrix `X_o.T @ X_o`.  What is the
relationship of these values to the *vector lengths* of the vectors in the
first and second columns of `X_o`?

Find the new value for $(\Xmat^T \Xmat)^{-1}$ – the inverse of `X_o.T @ X_o`.

In [31]:
iXtX_o = npl.inv(XtX_o)
# Show the result
iXtX_o

What is the relationship of the values in the diagonal of the inverse matrix to
the lengths of the vectors in the first and second columns of `X_o`?

*Hint*: $A^{-1} \cdot A = I$; if $A$ has all zeros off the diagonal, what must
$A^{-1}$ be for this to be true?

Make a new data vector `y_c` by subtracting the mean from the psychopathy
vector:

In [32]:
#- Make mean-centered version of psychopathy vector
y_c = psychopathy - np.mean(psychopathy)
# Show the result
y_c

Calculate a new `B_o` parameter vector for the least-squares fit of `X_o`
to `y_c`:

In [33]:
#- Calculate fit of X_o to y_c
B_o = iXtX_o @ X_o.T @ y_c
# Show the result
B_o

The first parameter has changed compared to your previous estimate.  Can you
explain why it has this new value by considering the values of $(\Xmat^T
\Xmat)^{-1} \Xmat^T \yvec$?

Calculate the correlation coefficient between `y_c` and the second column of
`X_o`.  In case you want to play it that way, here's a function to calculate
z-scores (standard scores), that you could use to calculate the correlation.

In [34]:
def standard_scores(v):
    return (v - np.mean(v)) / np.std(v)

In [35]:
#- Correlation coefficient of the second column of X_o and y_c
r_xy = np.mean(standard_scores(x_1) * standard_scores(y_c))
# Show the result
r_xy

Next we explore the relationship between this correlation coefficient and `x_1`
— the second column of the design matrix.

First - consider the calculation that gave us the value for the slope:

In [36]:
# The regression slope.
b = B_o[1]
b

We got this value via:

In [37]:
B_o = npl.inv(X_o.T @ X_o) @ X_o.T @ y_c
B_o

We can break this calculation up into:

In [38]:
# First part.
iXtX_o = npl.inv(X_o.T @ X_o)
# Second part
XtY_o = X_o.T @ y_c
# Result (B_o)
iXtX_o @ XtY_o

Consider the second part of the calculation:

In [39]:
XtY_o

This has two values:

* `B_o[0]`: the dot product of the vector of ones with `y_c`
* `B_o[1]`: the dot product of `x_1` with `y_c`

Now consider the first part of the calculation:

In [40]:
iXtX_o

Remember, this is just the inverse of:

In [41]:
XtX_o = X_o.T @ X_o
XtX_o

Matrix multiplication (`@`) takes the dot products of each left row with each
right column.  As you may have noticed above, this means the elements on the
diagonal are just the vector lengths of each column.  Because the columns are
orthogonal, the off-diagonal elements are all 0.

The [inverse of a diagonal matrix](https://textbook.nipraxis.org/diag_inverse)
is another diagonal matrix where the diagonal elements are given by the
reciprocals of the diagonal elements in the original array:

In [42]:
print('First diagonal element', 1 / XtX_o[0, 0])
print('Second diagonal element', 1 / XtX_o[1, 1])

Remember `x_1` is the second column of `X_o`.

Now consider the whole calculation that leads up to `B_o`.  Reproduce the
calculation of `B_o[1]`, without using matrix multiplication, and given:

* the vector length of `x_1`
* the dot product of `x_1` and `y_c`

Here is the length:

In [43]:
x_1_len = np.sqrt(x_1 @ x_1)
x_1_len

Here's the dot product:

In [44]:
x_dot_y = x_1 @ y_c
x_dot_y

In [45]:
b_reproduced = 1 / x_1_len ** 2 * x_dot_y
# Show the result
b_reproduced

Now we return to the correlation coefficient.

Remember this is given by first subtracting the mean from each vector, then
multiplying the resulting mean-centered vectors, to give the elementwise
product, then taking the mean of the elementwise product.

Remember that `x_1` and `x_c` already have mean zero.

Consider the calculation of the correlation coefficient, and that of the dot
product `x_dot_y` above.

Reconstruct `x_dot_y` above from the correlation coefficient, `n`,
`np.std(y_c)` and `np.std(x_1)`:

In [46]:
y_c_std = np.std(y_c)
x_dot_y_reconstructed = r_xy * n * y_c_std * x_1_std
# Show the result
x_dot_y_reconstructed

Remember the relationships you found above:

* Between the standard deviation and the vector length.
* Between the dot product of two zero-mean vectors and the correlation.

See if you can use a reworking of your formula above to reconstruct `B_o[1]`
with a calculation using only:

* `r_xy` (the correlation)
* `x_1_std`
* `y_c_std`
* `n` (if you need it).

Try simplifying the calculation as far as possible.

In [47]:
b_reproduced_again = r_xy *  y_c_std / x_1_std
# Show the result
b_reproduced_again

Did you get the same calculation as you saw in [on
regression](https://textbook.nipraxis.org/on_regression)?

## Different models, different parameters

Now try calculating $\bvec$ fitting the `X_o` design to the original
psychopathy data (not the mean-centered version).

In [48]:
#- Fit X_o to psychopathy data
B_o = iXtX @ X_o.T @ psychopathy
# Show the result
B_o

Compare the first value in the new `B_o` parameter vector with the mean of
the `psychpathy` vector.

In [49]:
np.mean(psychopathy)

Can you explain the relationship?

For extra points, can you explain why the second value in `B_o` did not
change when we estimated for `psychopathy` rather than the mean-centered
version `y_c`?  Hint: remember $(\vec{a} + \vec{b}) \cdot \vec{c} = \vec{a}
\cdot \vec{c} + \vec{b} \cdot \vec{c}$.

Calculate the fitted values for the `X_o` model, and compare them to the
fitted values for the original model:

In [50]:
fitted_X_o = X_o @ B_o
np.allclose(fitted_X_o, fitted)

For even more extra points, explain the relationship between the fitted values
for the original model and those for the new model, where the clammy regressor
is mean centered.